In [1]:
import numpy as np

In [2]:
s = np.loadtxt('new_Protein_edges_full.txt')   #edge file

In [5]:
x = s[:,0]
y = s[:,1]
max = np.max(x).astype(np.uint32)

In [6]:
c = 0
for i in range(max+1):
  if i not in x and i not in y:
    c = c+1

In [7]:
nodes = np.zeros((max+1-c,))
f =0
for i in range(max+1):
  if i in x or i in y:
    nodes[f] = i
    f = f+1

In [8]:
nodes = nodes.astype(np.uint32)

In [9]:
label1 = np.loadtxt('PROTEINS_full_node_labels.txt')     #label file

In [10]:
label1 = label1.astype(np.uint32)

In [11]:
labels = {}
for i in range(max+1):
  if i in nodes:
    labels[i] = label1[i] 

In [12]:
s = s.astype(np.uint32)

In [13]:
adj_list = {}
for i in range(nodes.shape[0]):
  adj_list[nodes[i]] = []
for i in range(1,s.shape[0]):
  adj_list[s[i][0]].append(s[i][1])
  #adj_list[s[i][1]].append(s[i][0])

In [15]:
r = np.loadtxt('Protein_full_128.emb')     #embedding file

In [ ]:
emb = {}
for i in range(nodes.shape[0]):
  emb[r[i,0]] = r[i,1:] 

In [ ]:
import random
c=0
neg_edges = np.zeros((300000,2))
while(c<300000):
  print(c)
  l = (random.sample(list(nodes[:]),1)[0], random.sample(list(nodes[:]),1)[0])
  neg_edges[c][0] = l[0]
  neg_edges[c][1] = l[1]
  c = c+1

In [ ]:
X_train = np.zeros((300000,256))
Y_train = np.zeros((300000,))
for i in range(300000):
  X_train[i] = np.concatenate((emb[neg_edges[i][0]],emb[neg_edges[i][1]]))
  if labels[neg_edges[i][0]] == labels[neg_edges[i][1]]:
    Y_train[i] = 1
  else:
    Y_train[i] = 0

In [ ]:
X_train_train = X_train[60000:,:]
X_train_test = X_train[:60000,:]
Y_train_train = Y_train[60000:]
Y_train_test = Y_train[:60000]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=100, random_state=0)
regressor.fit(X_train_train, Y_train_train)
y_pred = regressor.predict(X_train_test)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_train_test, y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_train_test, y_pred))